In [ ]:
#!pip install pomegranate

In [ ]:
#!pip install pymc3

In [ ]:
#!pip install graphviz

In [ ]:
#!pip install pygraphviz

In [ ]:
#!pip install matplotlib

In [ ]:
#!pip install openpyxl

In [ ]:
#!pip install sklearn

In [1]:
#imports
import pandas as pd
import numpy as np
import arviz as az
import pymc3 as pm
import matplotlib.pyplot as plt
from pomegranate import *

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\powel\Anaconda3\lib\site-packages\theano\configdefaults.py:697: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  "DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
#read in data
pd.set_option('display.max_columns', None)

#name format is df survey#_wave# if resend, add (_2 at end)

df1_1 = pd.read_csv("survey1.1.csv", encoding='cp1252')
df1_1_2 = pd.read_csv("survey1.1.2.csv", encoding='cp1252')
df1_2 = pd.read_csv("survey1.2.csv", encoding ='cp1252')
#survey 1 wave 3
#survey1.3.csv
#df1_3 = pd.read_csv...
df2_1 = pd.read_csv("survey2.1.csv", encoding='cp1252')
df2_1_2 = pd.read_csv("survey2.1.2.csv", encoding='cp1252')
df2_2 = pd.read_csv("survey2.2.csv", encoding='cp1252')
df3_1 = pd.read_csv("survey3.1.csv", encoding='cp1252')
print(df2_2.shape)

(333, 70)


In [3]:
survey1 =pd.concat([df1_1, df1_1_2, df1_2]) #add df1_3
survey2 = pd.concat([df2_1, df2_1_2, df2_2])
survey3 = pd.concat([df3_1])
survey1['survey']=np.full(survey1.shape[0], 1)
survey2['survey']=np.full(survey2.shape[0], 2)
survey3['survey']=np.full(survey3.shape[0], 3)

main = pd.concat([survey1, survey2, survey3])
main

,Date (Responses to recipient forms),Recipient,Origin (Responses to recipient forms),Score (Responses to recipient forms),Primary key,Conditional: Portal Engagement Explanation,Caregiver for Active Duty Military (Constituent Type),Caregiver for Veteran (Constituent Type),Other caregiver (Constituent Type),Veteran (Constituent Type),Other (Constituent Type),Sex,Age,Race:,Time Providing Care,Use of Video Conferencing,Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,Consent,Flow,Display and Format,Content,survey
0,2020/12/04 10:55:54,Evans Melanie (mryumyumsbakery@gmail.com),NaN,15,397637986,NaN,0,1,0,0,0,Female,45 - 54,White,5+ years,1+ Year,Yes,No,Neutral,Agree,Disagree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neutral,Strongly Disagree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Rarely,Sometimes,Never,Never,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Sometimes,1,0,0,1,0,0,1,1,0,1,0,0,0,0,1,1,1,0,0,1,1,1,0,Agree,1,5,5,5,1
1,2020/12/04 10:56:29,Miranda Jennifer (jmiranda26@icloud.com),NaN,14,397628088,NaN,0,1,0,0,0,Female,35 - 44,White,1 to 2 years,<3 Months,No,No,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Disagree,Strongly Disagree,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Agree,Agree,Agree,Neutral,Sometimes,Sometimes,Sometimes,Rarely,Never,Never,Never,Never,Rarely,Rarely,0,0,0,1,1,0,1,0,0,0,0,0,1,0,1,1,1,0,0,1,1,0,0,Agree,1,5,5,4,1
2,2020/12/04 10:56:42,Zuniga Adam (zuniga.adam77@gmail.com),NaN,15,397642487,NaN,0,0,0,1,0,Male,35 - 44,White,5+ years,3 To 6 Months,No,No,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Strongly Agree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Neutral,Agree,Agree,Never,Never,Rarely,Never,Never,Never,Never,Never,Never,Never,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,Agree,1,5,5,5,1
3,2020/12/04 10:57:02,Black Thomas (andrearuszala@gmail.com),NaN,12,397637989,NaN,0,0,0,1,0,Male,65+,White,6mo to 1year,6 To 9 Months,Yes,Yes,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Agree,Agree,Agree,Agree,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,Agree,1,4,4,4,1
4,2020/12/04 10:57:09,Jesseapontejr@Gmail.Com Jesse (jesseapontejr@g...,NaN,15,397642488,NaN,0,0,0,1,0,Male,35 - 44,Black or African American,2 to 3 years,1+ Year,

In [4]:
print(survey1.shape)
print(survey2.shape)
print(survey3.shape)

(702, 71)
(605, 71)
(12, 71)


In [5]:
toDrop = ['Date (Responses to recipient forms)', 'Recipient', 'Origin (Responses to recipient forms)', 'Primary key', 'Conditional: Portal Engagement Explanation', 'Consent', 'Flow', 'Display and Format', 'Content',]
main = main.drop(columns = toDrop)
#main.drop(list(main.filter(regex = 'Constituent Type')), axis = 1, inplace = True)
#main.drop(list(main.filter(regex = 'Chronic Conditions')), axis = 1, inplace = True)
#main.drop(list(main.filter(regex = 'Portal Video Use')), axis = 1, inplace = True)
#main.drop(list(main.filter(regex = 'Connection Technology Use')), axis = 1, inplace = True)

In [6]:
encoding = {
    'Agree': 1,
    'StronglyAgree': 2,
    'Strongly Agree': 2,
    'StronglyDisagree': -2,
    'Strongly Disagree': -2,
    'Disagree': -1,
    'Neutral': 0,
    'Yes': 1,
    'No': -1,
    'Rarely':-1,
    'Never':-2,
    'Sometimes':1,
    'Always':2,
    '<6mo':-3,
    '6mo to 1year':-2,
    '1 to 2 years': -1,
    '2 to 3 years': 0,
    '3 to 4 years':1,
    '4-5 years': 2,
    '5+ years': 3, 
    '<3 Months': -2,
    '3 To 6 Months': -1,
    '6 To 9 Months': 0,
    '9 To 12 Months': 1,
    '1+ Year': 2,
    '0 - 17': -3,
    '18 - 24': -2,
    '25 - 34': -1,
    '35 - 44': 0,
    '45 - 54': 1,
    '55 - 64': 2,
    '65+': 3,
    'Male': 0,
    'Female': 1,
    'White': -2, 
    'Black or African American': -1,
    'Asian': 0,
    'American Indian or Alaska Native': 1,
    'Some other race': 2,
    'Native Hawaiian and Pacific Islander': 3
    
}
def f(s):
    l = []
    for b in s:
        if (b in encoding):
            l.append(encoding[b])
        elif isinstance(b, str) and b.isnumeric():
            l.append(int(b))
        else:
            l.append(b)
    return pd.Series(l)


In [7]:
data = main.apply(f, axis = 0)
data = data.sample(frac=1).reset_index(drop=True)
data

,Score (Responses to recipient forms),Caregiver for Active Duty Military (Constituent Type),Caregiver for Veteran (Constituent Type),Other caregiver (Constituent Type),Veteran (Constituent Type),Other (Constituent Type),Sex,Age,Race:,Time Providing Care,Use of Video Conferencing,Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,survey
0,9,0,1,1,1,1,1,2,2,3,2,1,1,0,2,0,2,2,2,2,2,2,2,0,0,0,0,2,2,1,2,-1,2,2,2,1,2,1,0,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,1,1,1,0,2
1,15,0,0,0,1,0,0,1,-1,-3,-2,-1,1,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,2,2,2,1,1,-1,-2,-1,-2,-2,-2,-2,2,-2,1,1,1,0,1,0,1,1,0,0,0,1,1,0,0,1,1,1,0,1,1,1,1,2,2
2,15,0,1,0,0,0,1,3,-2,3,2,1,-1,1,1,1,1,0,1,1,0,1,0,1,0,1,1,-2,-1,-2,-2,-2,-2,-2,-2,-2,-2,0,1,1,1,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,2
3,15,0,1,0,0,0,1,3,-2,3,2,1,1,1,1,1,1,-1,0,1,1,0,0,1,0,-1,0,-1,-1,-2,-1,1,1,2,-1,2,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,2
4,12,0,0,0,1,0,0,0,-2,0,-1,1,1,0,1,2,2,1,0,0,0,0,-1,1,1,0,1,1,1,1,2,2,1,-1,2,2,2,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,11,0,1,0,0,0,1,0,-2,3,2,1,1,0,1,2,2,2,2,2,2,2,1,1,1,1,1,1,1,-1,1,1,1,1,-1,1,2,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,1,1,1,0,1,1,0,1,1,1
1315,15,0,1,0,0,0,1,0,2,3,1,1,-1,0,0,-1,1,0,-1,1,0,1,1,2,1,2,1,-1,1,-1,-1,1,-2,-1,1,1,1,1,0,1,1,1,1,1,0,0,0,0,0,1,0,1,1,1,0,0,1,1,1,0,2,1
1316,15,0,0,0,1,0,0,1,-1,3,2,1,1,-1,1,1,1,-1,-1,1,1,0,1,1,1,1,1,2,1,1,1,1,1,-1,1,1,1,0,0,0,1,1,0,1,0,0,0,0,0,1,0,1,1,1,0,0,1,1,0,0,1,2
1317,15,0,0,0,1,0,0,3,-2,-3,-1,1,1,-1,1,-1,1,2,-1,1,0,1,-1,1,0,1,1,1,-1,-1,2,-2,-1,1,1,-1,1,0,0,0,1,1,1,0,1,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,1,1


In [8]:
#ls = []
#for column in data:
#    ls.append(data[column].unique())
#ls

In [9]:
data1 = data[data['survey']==1]
data2 = data[data['survey']==2]

In [10]:
#data1.sum(axis=0)/702

In [11]:
pd.set_option('display.max_rows', 100)

In [75]:
temp = data.copy()
temp.drop(list(main.filter(regex = 'Chronic Conditions')), axis = 1, inplace = True)
temp.drop(list(main.filter(regex = 'Constituent Type')), axis = 1, inplace = True)
temp = temp.drop(columns = ['Sex', 'Age', 'Race:', 'survey', 'Time Providing Care'])
temp

,Score (Responses to recipient forms),Use of Video Conferencing,Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help
0,9,2,1,1,0,2,0,2,2,2,2,2,2,2,0,0,0,0,2,2,1,2,-1,2,2,2,1,2,1,0,0,0,0,1,1,1,1,0
1,15,-2,-1,1,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,2,2,2,1,1,-1,-2,-1,-2,-2,-2,-2,2,-2,0,1,1,1,0,1,1,1,1,2
2,15,2,1,-1,1,1,1,1,0,1,1,0,1,0,1,0,1,1,-2,-1,-2,-2,-2,-2,-2,-2,-2,-2,1,0,0,0,0,0,0,0,1,1
3,15,2,1,1,1,1,1,1,-1,0,1,1,0,0,1,0,-1,0,-1,-1,-2,-1,1,1,2,-1,2,-1,0,1,1,0,0,0,0,0,1,0
4,12,-1,1,1,0,1,2,2,1,0,0,0,0,-1,1,1,0,1,1,1,1,2,2,1,-1,2,2,2,0,1,1,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,11,2,1,1,0,1,2,2,2,2,2,2,2,1,1,1,1,1,1,1,-1,1,1,1,1,-1,1,2,0,1,1,1,0,1,1,0,1,1
1315,15,1,1,-1,0,0,-1,1,0,-1,1,0,1,1,2,1,2,1,-1,1,-1,-1,1,-2,-1,1,1,1,1,1,1,0,0,1,1,1,0,2
1316,15,2,1,1,-1,1,1,1,-1,-1,1,1,0,1,1,1,1,1,2,1,1,1,1,1,-1,1,1,1,1,1,1,0,0,1,1,0,0,1
1317,15,-1,1,1,-1,1,-1,1,2,-1,1,0,1,-1,1,0,1,1,1,-1,-1,2,-2,-1,1,1,-1,1,0,1,1,0,0,0,1,0,0,1


In [78]:
hred = ['Portal Experience', 'Portal Engagement', 'Portal Engagement Reinforcement', 'Portal Wellbeing Influence', 'Use of Video Conferencing', 'Caregivers (Portal Video Use)', 'Family members (Portal Video Use)','Friends (Portal Video Use)', 'Others (Portal Video Use)', 'SMS/text (Connection Technology Use)','Phone (Connection Technology Use)','Video Portal (Connection Technology Use)','Other (Connection Technology Use)','Portal Chronic Disease Management Help'] 

In [79]:
def custom_style(row):

    color = ''
    if row.values[0] in hred and row.values[-1] == "Alternative accepted: mean is smaller":
        color = 'lime'
    if (not row.values[0] in hred) and row.values[-1] == "Alternative accepted: mean is larger":
        color = 'lime'
    if (not row.values[0] in hred) and row.values[-1] == "Alternative accepted: mean is smaller":
        color = 'red'
    if row.values[0] in hred and row.values[-1] == "Alternative accepted: mean is larger":
        color = 'red'

    return ['background-color: %s' % color]*len(row.values)

In [80]:
def stats(data1, data2):
    results = []

    for col in w1data.columns: 
        vals1 = data1[col].to_list()
        vals2 = data2[col].to_list()
    
    # Null hypothesis: means are the same
    # Alternative: the mean after is higher
    
        cm = ssw.CompareMeans(ssw.DescrStatsW(vals1), ssw.DescrStatsW(vals2))
        res = cm.ztest_ind(alternative='smaller',usevar='unequal')
        msg = "Cannot reject the null"
        alpha = 0.05/39
        if (res[1] <= alpha):
            msg = "Alternative accepted: mean is larger"
       
        elif (res[1] >= 1-alpha):
            msg = "Alternative accepted: mean is smaller"
    
        if (res[1] <= 0.5):
            results.append([col, res[1]])
        else:
            results.append([col, 1-res[1]])

    res = pd.DataFrame(results, columns=['Item', 'P-Value'])
    return red #res.style.apply(custom_style, axis=1)

In [81]:
w1data = temp[data['survey']==1]
w2data = temp[data['survey']==2]

import statsmodels.stats.weightstats as ssw

stats(w1data, w2data)

,Item,P-Value,Verdict
0,Score (Responses to recipient forms),0.000000,Alternative accepted: mean is larger
1,Use of Video Conferencing,0.000000,Alternative accepted: mean is smaller
2,Depression,0.402271,Cannot reject the null
3,Motivation,0.226219,Cannot reject the null
4,Life Satisfaction,0.002105,Cannot reject the null
5,Physical Exhaustion,0.020881,Cannot reject the null
6,Desire To Escape,0.050746,Cannot reject the null
7,Same Self,0.096624,Cannot reject the null
8,Financial Decrease,0.031210,Cannot reject the null
9,Health Change,0.001292,Cannot reject the null


In [73]:
def stat3(turn, c):
    print(c)
    a1data = temp[data[c]<= turn]
    a2data = temp[data[c]>= turn+1]

    display(stats(a1data, a2data))
    
    b1data = a1data.loc[(data['survey'] == 1)]
    b2data = a1data.loc[(data['survey'] == 2)]
 
    display(stats(b1data, b2data))
    
    c1data = a2data.loc[(data['survey'] == 1)]
    c2data = a2data.loc[(data['survey'] == 2)]

    display(stats(c1data, c2data))

In [82]:
thing = ['Hypertension (high blood pressure) (Chronic Conditions)','Hyperlipidemia (high blood cholesterol or triglyceride levels)  (Chronic Conditions)','Allergies, sinusitis and other upper respiratory conditions  (Chronic Conditions)','Arthritis (Chronic Conditions)','Mood Disorder (depression and bipolar disorder) (Chronic Conditions)','Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions)','Diabetes (Type1 and Type 2)  (Chronic Conditions)','Anxiety Disorder (Chronic Conditions)','Asthma (Chronic Conditions)','Thyroid disorder (Chronic Conditions)','Traumatic Brain Injury (TBI) (Chronic Conditions)']
choice = thing[1] #change this to change what we're analysing
choice = 'Traumatic Brain Injury (TBI) (Chronic Conditions)'
stat3(0, 'Age')
stat3(0, 'Veteran (Constituent Type)')
stat3(0, 'Sex')
stat3(-2, 'Race:')

for choice in thing:
    stat3(0, choice)


Age


,Item,P-Value,Verdict
0,Score (Responses to recipient forms),0.043175,Cannot reject the null
1,Use of Video Conferencing,0.000092,Alternative accepted: mean is smaller
2,Depression,0.074895,Cannot reject the null
3,Motivation,0.017174,Cannot reject the null
4,Life Satisfaction,0.125219,Cannot reject the null
5,Physical Exhaustion,0.115920,Cannot reject the null
6,Desire To Escape,0.061544,Cannot reject the null
7,Same Self,0.229178,Cannot reject the null
8,Financial Decrease,0.323363,Cannot reject the null
9,Health Change,0.007582,Cannot reject the null


,Item,P-Value,Verdict
0,Score (Responses to recipient forms),0.000058,Alternative accepted: mean is larger
1,Use of Video Conferencing,0.008570,Cannot reject the null
2,Depression,0.189916,Cannot reject the null
3,Motivation,0.265206,Cannot reject the null
4,Life Satisfaction,0.000011,Alternative accepted: mean is larger
5,Physical Exhaustion,0.000002,Alternative accepted: mean is larger
6,Desire To Escape,0.002338,Cannot reject the null
7,Same Self,0.000052,Alternative accepted: mean is larger
8,Financial Decrease,0.001271,Alternative accepted: mean is larger
9,Health Change,0.000027,Alternative accepted: mean is larger


,Item,P-Value,Verdict
0,Score (Responses to recipient forms),0.000000,Alternative accepted: mean is larger
1,Use of Video Conferencing,0.000000,Alternative accepted: mean is smaller
2,Depression,0.486781,Cannot reject the null
3,Motivation,0.161016,Cannot reject the null
4,Life Satisfaction,0.442604,Cannot reject the null
5,Physical Exhaustion,0.281562,Cannot reject the null
6,Desire To Escape,0.314100,Cannot reject the null
7,Same Self,0.125381,Cannot reject the null
8,Financial Decrease,0.429779,Cannot reject the null
9,Health Change,0.476956,Cannot reject the null


Race:


...

,Item,P-Value,Verdict
0,Score (Responses to recipient forms),0.004887,Cannot reject the null
1,Use of Video Conferencing,0.003372,Cannot reject the null
2,Depression,0.008431,Cannot reject the null
3,Motivation,0.055079,Cannot reject the null
4,Life Satisfaction,0.070396,Cannot reject the null
5,Physical Exhaustion,0.064102,Cannot reject the null
6,Desire To Escape,0.396753,Cannot reject the null
7,Same Self,0.427705,Cannot reject the null
8,Financial Decrease,0.398400,Cannot reject the null
9,Health Change,0.065207,Cannot reject the null


Traumatic Brain Injury (TBI) (Chronic Conditions)


,Item,P-Value,Verdict
0,Score (Responses to recipient forms),0.323471,Cannot reject the null
1,Use of Video Conferencing,0.000279,Alternative accepted: mean is larger
2,Depression,0.011115,Cannot reject the null
3,Motivation,0.000011,Alternative accepted: mean is larger
4,Life Satisfaction,0.036086,Cannot reject the null
5,Physical Exhaustion,0.046343,Cannot reject the null
6,Desire To Escape,0.087831,Cannot reject the null
7,Same Self,0.002145,Cannot reject the null
8,Financial Decrease,0.370773,Cannot reject the null
9,Health Change,0.470413,Cannot reject the null


,Item,P-Value,Verdict
0,Score (Responses to recipient forms),0.000000,Alternative accepted: mean is larger
1,Use of Video Conferencing,0.000000,Alternative accepted: mean is smaller
2,Depression,0.306186,Cannot reject the null
3,Motivation,0.226218,Cannot reject the null
4,Life Satisfaction,0.010176,Cannot reject the null
5,Physical Exhaustion,0.081159,Cannot reject the null
6,Desire To Escape,0.121131,Cannot reject the null
7,Same Self,0.312996,Cannot reject the null
8,Financial Decrease,0.131842,Cannot reject the null
9,Health Change,0.017755,Cannot reject the null


,Item,P-Value,Verdict
0,Score (Responses to recipient forms),0.006324,Cannot reject the null
1,Use of Video Conferencing,0.000742,Alternative accepted: mean is smaller
2,Depression,0.493745,Cannot reject the null
3,Motivation,0.359062,Cannot reject the null
4,Life Satisfaction,0.025844,Cannot reject the null
5,Physical Exhaustion,0.034250,Cannot reject the null
6,Desire To Escape,0.081082,Cannot reject the null
7,Same Self,0.019791,Cannot reject the null
8,Financial Decrease,0.058636,Cannot reject the null
9,Health Change,0.013259,Cannot reject the null


##Ignore everything past this

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
c1data['Depression'].shape[0]

In [ ]:
sum(c1data['Depression'].value_counts().tolist())

In [ ]:

def graph1(labels, values, vlabels, xlabel, ylabel, title):

    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    ax.bar(x, values[0], width, label= vlabels[0])

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    fig.tight_layout()

    plt.show()

def graph2(labels, values, vlabels, xlabel, ylabel, title):

    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    ax.bar(x - width/2, values[0], width, label= vlabels[0])
    ax.bar(x + width/2, values[1], width, label= vlabels[1])

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation = 15)
    ax.legend()

    fig.tight_layout()

    plt.show()

def graph3(labels, values, vlabels, xlabel, ylabel, title):

    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    ax.bar(x - width, values[0], width, label= vlabels[0])
    ax.bar(x, values[1], width, label= vlabels[1])
    ax.bar(x + width, values[2], width, label= vlabels[2])

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    fig.tight_layout()

    plt.show()

In [ ]:
a = [1, 2, 3]

In [ ]:
np.array(a) / 2


In [ ]:
labels = ['Caretaker (Any)', 'Military Caregiver', 'Veteran Caregiver', 'Veterans', 'Neither']
values1 = [429, 11, 417, 294, 5]
values2 = [399, 9, 376, 241, 5]
graph2(labels, [np.array(values1)/702, np.array(values2)/605], ['Survey 1', 'Survey 2'], 'Role', 'Frequency', 'Relative Role Distribution')
graph2(labels, [np.array(values1), np.array(values2)], ['Survey 1', 'Survey 2'], 'Role', 'Frequency', 'Role Distribution')

In [ ]:
labels = ['Depressed', 'Not Depressed']
graph1(labels, [c1data['Depression'].value_counts().tolist()], ['c1'], '', 'Frequency', 'Depression Distribution')

In [ ]:
pd.set_option('display.max_rows', 100)
data.sort_values(by=['survey'])

In [ ]:
#toDrop2 = ['Depression', 'Motivation', 'Physical Exhaustion', 'Desire To Escape', 'Same Self', 'Financial Decrease', 'Health Change', 'Demands Strength', 'Competing Demand', 'Concern About Future', 'Relationship Impact', 'Unhappy Alone', 'No One To Talk To', 'Cant Tolerate Being Alone', 'No One Understands Me', 'Waiting for Contact', 'Feel Completely Alone', 'Unable To Communicate', 'Need Company', 'Difficult to make friends', 'Feel Excluded']
toDrop3 = ['Score (Responses to recipient forms)']
data1 = data.drop(columns = toDrop2)
data1 = data.drop(columns = toDrop3)
data1.shape

In [ ]:
test1 = data[data['survey']==1]['Use of Video Conferencing']
test2 = data[data['survey']==2]['Use of Video Conferencing']
print(test1.sum()/test1.shape[0])
print(test2.sum()/test2.shape[0])

In [ ]:
train_size = int(data1.shape[0]*0.8)
test_size = int(data1.shape[0]*0.2)

In [ ]:
#creates Bayesian model using data

dtrain = data1[:train_size]
dtest = data1[:-test_size]

In [ ]:
model = BayesianNetwork.from_samples(dtrain)

In [ ]:
blanked = dtest.copy()
blanked['Score (Responses to recipient forms)'] = np.full(blanked.shape[0], None)
prediction = model.predict(blanked.values.tolist())
mean = dtest["Score (Responses to recipient forms)"].mean()
rss = 0
tss = 0
correct = 0
for i in range (0, dtest.shape[0]):
    yBar = prediction[i][0]
    
    yActual = dtest['Score (Responses to recipient forms)'][i]
    
    if (yBar == yActual):
        correct+=1
    
    rss += (yActual-yBar)**2
    tss += (yActual-mean)**2

percent = float(correct) / dtest.shape[0]
r2 = 1-float(rss)/tss
print("Accuracy: " + (str)(percent))
print("r^2: " + (str)(r2))


from sklearn.metrics import precision_score, recall_score, accuracy_score, r2_score

labels = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
y_true = dtest['Score (Responses to recipient forms)']
y_pred = [row[0] for row in prediction]

precision = precision_score(y_true, y_pred, labels=labels, average='micro')
recall = recall_score(y_true, y_pred, labels=labels, average='micro')
accuracy = accuracy_score(y_true, y_pred)
# Need to make y_true shape (samples, classes)

#y_true = y_true.values.reshape((y_true.shape[0],len(labels)))
#y_pred = y_pred.reshape((y_pred.shape[0],len(labels)))

r_squared = r2_score(y_true, y_pred)
#auc = roc_auc_score(y_true, y_pred)

print('Accuracy: %.3f' % accuracy)
print('R^2: %.3f' % r_squared)
print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)
#print('ROC AUC: %.3f' % auc)

In [ ]:
blanked = dtest.copy()
blanked['Life Satisfaction'] = np.full(blanked.shape[0], None)
prediction = model.predict(blanked.values.tolist())
mean = dtest["Life Satisfaction"].mean()
rss = 0
tss = 0
correct = 0
for i in range (0, dtest.shape[0]):
    yBar = prediction[i][0]
    
    yActual = dtest['Life Satisfaction'][i]
    print(yBar)
    if (yBar == yActual):
        correct+=1
    
    rss += (yActual-yBar)**2
    tss += (yActual-mean)**2

percent = float(correct) / dtest.shape[0]
r2 = 1-float(rss)/tss
print("Accuracy: " + (str)(percent))
print("r^2: " + (str)(r2))


from sklearn.metrics import precision_score, recall_score, accuracy_score, r2_score

labels = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
y_true = dtest['Life Satisfaction']
y_pred = [row[0] for row in prediction]

precision = precision_score(y_true, y_pred, labels=labels, average='micro')
recall = recall_score(y_true, y_pred, labels=labels, average='micro')
accuracy = accuracy_score(y_true, y_pred)
# Need to make y_true shape (samples, classes)

#y_true = y_true.values.reshape((y_true.shape[0],len(labels)))
#y_pred = y_pred.reshape((y_pred.shape[0],len(labels)))

r_squared = r2_score(y_true, y_pred)
#auc = roc_auc_score(y_true, y_pred)

print('Accuracy: %.3f' % accuracy)
print('R^2: %.3f' % r_squared)
print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)
#print('ROC AUC: %.3f' % auc)

In [ ]:
from pymc3.glm import GLM

with pm.Model() as model_glm:
    GLM.from_formula("Life_Satisfaction ~ Caregiver_for_Veteran + Other_caregiver + Veteran + Other + Hypertension_high_blood_pressure_Chronic_Conditions + Hyperlipidemia_high_blood_cholesterol_or_triglyceride_levels_Chronic_Conditions + Allergies + sinusitis_and_other_upper_respiratory_conditions_Chronic_Conditions + Arthritis_Chronic_Conditions + Mood_Disorder_depression_and_bipolar_disorder_Chronic_Conditions + Diabetes_Type1_and_Type_2_Chronic_Conditions + Anxiety_Disorder_Chronic_Conditions + Asthma_Chronic_Conditions + Coronary_artery_disease_includes_myocardial_infarction_heart_attack_Chronic_Conditions + Thyroid_disorder_Chronic_Conditions + Chronic_obstructive_lung_disease_and_bronchiectasis_Chronic_Condition + Traumatic_Brain_Injury__TBI_Chronic_Conditions + Post_Traumatic_Stress_Disorder_PTSD_Chronic_Conditions + None_of_the_above__Chronic_Conditions + Caregivers_Portal_Video_Use + Family_members__Portal_Video_Us + Friends_Portal_Video_Use + Others_Portal_Video_Use + Not_using_Portal_video_Portal_Video_Use + Portal_Chronic_Disease_Management_Help + SMS_text_Connection_Technology_Use + Phone_Connection_Technology_Use + Video_Portal_Connection_Technology_Use + Other_Connection_Technology_Use + before", dtrain)
    trace = pm.sample()

In [ ]:
with model_glm:
    axes_arr = az.plot_trace(trace)
plt.draw()

In [ ]:
blanked = dtest.copy(dtest)
blanked['Life_Satisfaction'] = np.full(blanked.shape[0])
prediction = model.predict(blanked)
mean = dtest["weight"].mean()
rss = 0
tss = 0
correct = 0
for i in range (0, dtest.shape[0]):
    yBar = prediction['Life_Satisfaction'][i]
    yActual = dtest['Life_Satisfaction'][i]
    if (yBar == yActual):
        correct+=1
    rss += (yActual-yBar)**2
    tss += (yActual-mean)**2
percent = (float)correct/200
r2 = 1-(float)rss/tss
print("Accuracy: " + (str)(percent))
print("r^2: " + (str)(r2))

from sklearn.metrics import precision_score, recall_score, accuracy_score, r2_score

labels = [-2, -1, 0, 1, 2]
y_true = dtest['Life_Satisfaction']
y_pred = prediction['Life_Satisfaction']

precision = precision_score(y_true, y_pred, labels=labels, average='micro')
recall = recall_score(y_true, y_pred, labels=labels, average='micro')
accuracy = accuracy_score(y_true, y_pred)
# Need to make y_true shape (samples, classes)

y_true = y_true.reshape((y_true.shape[0],len(labels)))
y_pred = y_pred.reshape((y_pred.shape[0],len(labels)))

r_squared = r2_score(y_true, y_pred)
auc = roc_auc_score(y_true, y_pred)

print('Accuracy: %.3f' % accuracy)
print('R^2: %.3f' % r_squared)
print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)
print('ROC AUC: %.3f' % auc)